# Eager Execution in Tensorflow 2.0

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

## Tensor Operations

In [ ]:
tf.add(1, 2)

In [ ]:
tf.add([1, 2], [3, 4])

In [ ]:
tf.square(5)

In [ ]:
tf.reduce_sum([1, 2, 3])

In [ ]:
tf.square(2) + tf.square(3)

In [ ]:
x = tf.matmul([[1]], [[2, 3]])

In [ ]:
x.shape

In [ ]:
x.dtype

In [ ]:
x.numpy()

## Custom Layers

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", 
                                      shape=[int(input_shape[-1]), 
                                             self.num_outputs],
                                      initializer='normal')
    
        self.bias = self.add_weight("bias", 
                                    shape=[self.num_outputs,],
                                    initializer='normal')

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel) + self.bias

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation

In [ ]:
model = Sequential([
    MyDenseLayer(20),
    Activation('relu'),
    MyDenseLayer(3),
])

In [ ]:
model.build(input_shape=(None, 4))

In [ ]:
model.summary()

In [ ]:
X = tf.random.normal([7, 4])

In [ ]:
X

### Model works like a function:

In [ ]:
model(X)

In [ ]:
df = pd.read_csv('../data/iris.csv')
df.head()

In [ ]:
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values.astype('float32')
y = df['species'].map({"setosa": 0, "versicolor": 1, "virginica": 2}).values.astype('int32')

### Datasets

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(1000).repeat().batch(16)

## One batch:

In [ ]:
dataset.take(1)
for data, labels in dataset.take(1):
    print(data)
    print(labels)
    print("Logits: ", model(data))

## Optimizer and Loss

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
optimizer = Adam()
loss = SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
logits = model(data)
logits

In [ ]:
loss(labels, logits).numpy()

## Gradients

In [ ]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        logits = model(data, training=True)
        loss_value = loss(labels, logits)
    return tape.gradient(loss_value, model.variables)

grad(model, data, labels)

In [ ]:
gw1, gb1, gw2, gb2  = grad(model, data, labels)

In [ ]:
plt.imshow(gw1);

In [ ]:
plt.imshow(gw2);

## Training Loop

In [ ]:
loss_history = []
accuracy_history = []
acc = tf.keras.metrics.SparseCategoricalAccuracy()

for (i, (data, labels)) in enumerate(dataset.take(400)):
    with tf.GradientTape() as tape:
        logits = model(data, training=True)
        loss_value = loss(labels, logits)

    loss_history.append(loss_value.numpy())
    
    grads = tape.gradient(loss_value, model.trainable_variables)
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
    acc.update_state(labels, logits)
    accuracy = acc.result().numpy()
    accuracy_history.append(accuracy)
    
    if i % 20 == 0:
        print(i, loss_value.numpy(), accuracy)

In [ ]:
plt.plot(loss_history)
plt.plot(accuracy_history)